In [11]:
from numba import njit
import numpy as np

@njit(fastmath=True)
def montecarlo_pi(n):
    count = 0
    for i in range(n):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        if x**2 + y**2 < 1:
            count += 1
    return 4 * count / n    

print(montecarlo_pi(100000000))

3.14158708


In [17]:
import numpy as np
import numba
from time import time

# Define a more complex operation in pure Python
def complex_operation_python(a, b):
    result = np.empty_like(a)
    for i in range(a.size):
        result[i] = a[i] * b[i] + a[i] - b[i]
    return result

# Define the same operation using Numba's @jit decorator
@numba.jit(nopython=True, fastmath=True)
def complex_operation_numba(a, b):
    result = np.empty_like(a)
    for i in range(a.size):
        result[i] = a[i] * b[i] + a[i] - b[i]
    return result

# Generate large random vectors for testing
vector_size = 100000000
a = np.random.rand(vector_size)
b = np.random.rand(vector_size)

# Measure the execution time of both functions
# Measure time for pure Python function
start_time = time()
result_python = complex_operation_python(a, b)
python_time = time() - start_time

# Run the Numba function once to trigger compilation
complex_operation_numba(a, b)

# Measure time for Numba-accelerated function
start_time = time()
result_numba = complex_operation_numba(a, b)
numba_time = time() - start_time

# Print the results
print(f"Pure Python time: {python_time:.6f} seconds")
print(f"Numba time: {numba_time:.6f} seconds")

Pure Python time: 42.110614 seconds
Numba time: 0.133030 seconds
